In [5]:
'''
Write a SciPy program that uses Rayleight-Sommerfield diffraction formula to calculate a diffraction pattern at 
a distance "far away" from rectangular and circular aperatus.

The patern should look something like the Fourier transform of the aperture functions!

The main idea is to write discrete version of Rayleight-Sommerfield diffraction and then calculate for 
our functions!
'''



'\nWrite a SciPy program that uses Rayleight-Sommerfield diffraction formula to calculate a diffraction pattern at \na distance "far away" from rectangular and circular aperatus.\n\nThe patern should look something like the Fourier transform of the aperture functions!\n'